In [ ]:
import pandas as pd
import json

import openai
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_api_key")

client = openai.OpenAI()

In [ ]:
# Load data from CSV files

base_plans = pd.read_csv("BasePlans.csv")
riders = pd.read_csv("Riders.csv")
junction = pd.read_csv("Junction.csv")
policy_holders = pd.read_csv("PolicyHolders.csv")
policy_riders = pd.read_csv("PolicyRiders.csv")

policy_holders = policy_holders.set_index("PolicyHolderID")
base_plans = base_plans.set_index("PolicyID")
riders = riders.set_index("RiderID")
policy_riders = policy_riders.set_index("PolicyHolderID")


In [ ]:
base_plans

In [ ]:
riders

In [ ]:
policy_holders

In [ ]:
policy_riders

We will define two functions that will become tools for our AdvisorBot.

**get_policy_data** returns all available policy rider options for the policy holders base plan. (It uses the function get_available_riders as part of its code).   
**check_current_coverage** returns the current riders the policy holder has at this time.

In [ ]:
def get_policy_data(user_id) :
    policy_holder = policy_holders.loc[user_id].Name
    rider_summary = get_available_riders(policy_holders.loc[user_id].BasePolicyID)

    policy_data = f"{policy_holder}:\n{rider_summary}"                 
    return policy_data


In [ ]:
def get_available_riders(base_policy_id):
    riders_for_base = junction[junction["PolicyID"] == base_policy_id]["RiderID"].tolist()
    rider_details = riders.loc[riders_for_base]
    
    rider_summary = "\n".join(
        f"- {row['RiderName']}: {row['Description']}"
        for _, row in rider_details.iterrows()
    )
    
    return f"Policy {base_plans.loc[base_policy_id].PolicyName} Riders:\n{rider_summary}"
    return rider_summary


In [ ]:
def check_current_coverage(user_id) :
    user_current_riders = pd.merge(policy_riders.loc[user_id]['RiderID'], riders, left_on = "RiderID", right_index = True).RiderName.tolist()
    return user_current_riders

In [ ]:
print(get_policy_data("PH020"))

In [ ]:
print(check_current_coverage("PH020"))

Here we have formatted the functions so that they can be used in the chat.completion.create() tools argument.

In [ ]:
check_current_coverage_tool = {
    "type": "function",
    "function": {
          "name": "check_current_coverage",
          "description": "Lookup the current riders for a specific policyholder ID",
          "parameters": {
                "type": "object",
                "properties": {
                      "user_id": {
                            "type": "string",
                            "description": "The unique ID of the policyholder for whom current riders are being retrieved."
                        }
                },
                "required": ["user_id"]
            }
    }
}

get_policy_data_tool = {
    "type": "function",
    "function": {
          "name": "get_policy_data",
          "description": "Lookup a policy holder's base policy and all available riders",
          "parameters": {
                "type": "object",
                "properties": {
                      "user_id": {
                            "type": "string",
                            "description": "The unique ID of the policyholder for whom available coverage is being retrieved."
                        }
                },
                "required": ["user_id"]
            }
    }
}

In [ ]:
def get_response_from_messages(messages_input, model_input = "gpt-4o-mini", temperature_input = .5):

    response = client.chat.completions.create(
        model = model_input,
        messages = messages_input,
        temperature = temperature_input,
        tools = [check_current_coverage_tool, get_policy_data_tool],
        tool_choice = "auto"
    )

    response = response
    return response 

In [ ]:
def initialize_bot():
    system_message = f"""
    You are AdvisorBot, an intelligent and empathetic assistant specializing in helping insurance policyholders
    understand their coverage and explore options for additional protection. Your primary role is to engage with policyholders to:

    Clarify Needs:
    Determine if they are asking about an incident that already happened and whether they are covered for it.
    Identify if they are seeking advice on adding new coverage due to changes in their lifestyle or circumstances.
    
    Access Tools:
    Use the check_current_coverage tool to fetch the current riders the policyholder has by asking for their policyholder ID.
    Use the get_policy_data tool to retrieve all available riders offered by the insurance company.

    Engage Empathetically:
    Actively listen to the user’s needs and ask relevant follow-up questions to understand their situation.
    Reassure them if they are concerned about an incident, and explain coverage options clearly.

    Provide Clear Recommendations:

    For incidents, check if the current riders cover the situation, and guide the user on the next steps if they are covered
    or suggest alternatives if they are not.
    For new coverage, recommend additional riders based on their described needs and the available options.  These riders must come from
    list of available riders retrieved using the get_policy_tool

    Efficient and Accurate:
    Ensure the user provides their policyholder ID to access their current coverage details and tailor recommendations
    specifically to their situation.
    Use the tools effectively to retrieve and analyze data to assist the user promptly.
    You should strive to make the interaction seamless and informative, ensuring the user feels supported and confident
    in their insurance decisions.
    """
    
    context = [{"role":"system", "content":system_message}]

    return context

In [ ]:
def collect_messages(prompt, context):
    context.append({'role':'user', 'content':prompt})
    response = get_response_from_messages(context) 
    context.append({'role':'assistant', 'content': f"{response.choices[0].message.content}"})
    return response

In [ ]:
def main():
    context = initialize_bot()
    total_tokens = 0

    print("Welcome! Type 'exit' to end the chat.")
    greeting = collect_messages("", context)
    print(greeting.choices[0].message.content)
    total_tokens += greeting.usage.total_tokens
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == 'exit':
            print(f"Total tokens used in this session: {total_tokens}")
            return context
        
        response = collect_messages(user_input, context)

        # Display the assistant's response if it has content
        if response.choices[0].message.content != None:
            print("Bot:", response.choices[0].message.content)
            total_tokens += response.usage.total_tokens

        # Process tool calls if included in response
        if response.choices[0].message.tool_calls:
            for tool_call in response.choices[0].message.tool_calls:
                tool_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)
                
                if tool_name == "check_current_coverage":
                    user_id = arguments["user_id"]
                    user_current_riders = check_current_coverage(user_id)
                    context.append({
                        'role': 'function', 'name': 'check_current_coverage','content': f"current coverage includes: {user_current_riders}"})
                
                if tool_name == "get_policy_data":
                    user_id = arguments["user_id"]
                    policy_data = get_policy_data(user_id)
                    context.append({
                        'role': 'function', 'name': 'get_policy_data','content': f"available policy options: {policy_data}"})
                
                #After processing each tool call, continue the conversation with updated context
                follow_up_response = collect_messages("", context)
                if follow_up_response.choices[0].message.content != None:
                    print("Bot:", follow_up_response.choices[0].message.content)
                    total_tokens += follow_up_response.usage.total_tokens


In [ ]:
chat_transcript = main()

In [ ]:
chat_transcript